# MNIST data classification: Keras and input pipeline

This notebook shows how to use input pipelines with Keras.

In [1]:
import numpy as np
import math
import tensorflow as tf

In [2]:
tf.VERSION

'1.12.0'

In [3]:
def decode(serialized_example):
    """Parses an image and label from the given `serialized_example`."""
    features = tf.parse_single_example(
        serialized_example,
        features={
            'image_raw': tf.FixedLenFeature([], tf.string),
            'label': tf.FixedLenFeature([], tf.int64),
        })
    label = tf.cast(features['label'], tf.int32)
    image = tf.decode_raw(features['image_raw'], tf.uint8)
    return image, label


def normalize(image, label):
    """Convert `image` from [0, 255] -> [-0.5, 0.5] floats."""
    image = tf.cast(image, tf.float32) * (1. / 255) - 0.5
    return image, label


def train_input_fn():
    """Reads input data num_epochs times."""
    dataset = tf.data.TFRecordDataset('train.tfrecords')
    dataset = dataset.map(decode)
    dataset = dataset.map(normalize)
    dataset = dataset.shuffle(6000)
    dataset = dataset.batch(256)
    dataset = dataset.repeat(5)
    return dataset

In [4]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(784,)),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [5]:
opt = tf.keras.optimizers.Adam(0.001)

In [6]:
model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
fit = model.fit(train_input_fn(),
                epochs=5,
                steps_per_epoch=math.ceil(60000 / 256))

Epoch 1/5
235/235 [==============================] - 5s 22ms/step - loss: 0.5075 - acc: 0.8563
Epoch 2/5
235/235 [==============================] - 5s 20ms/step - loss: 0.2744 - acc: 0.9207
Epoch 3/5
235/235 [==============================] - 5s 23ms/step - loss: 0.2194 - acc: 0.9368
Epoch 4/5
235/235 [==============================] - 5s 21ms/step - loss: 0.1814 - acc: 0.9482
Epoch 5/5
235/235 [==============================] - 5s 22ms/step - loss: 0.1538 - acc: 0.9559
